# Example for using SCD Transformer and Compute with ConDynS

We demonstrate here how to use SCD Transformer for writing SCDs with your custom prompts. Then, we show how to compute ConDynS.

In [1]:
import os
from convokit import Corpus, download
from convokit.convo_similarity import SCD
from convokit.convo_similarity.condyns import ConDynS
from convokit.genai import GenAIConfigManager

2025-10-02 03:47:14.871250: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-02 03:47:14.950271: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-02 03:47:16.470796: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
corpus = Corpus(filename=download("friends-corpus"))

Dataset already exists at /reef/kz88/convokit/download_corpus/friends-corpus


## Write SCD and SoP with SCD Transformer

In [ ]:
### Config your GenAI API keys
config = GenAIConfigManager()

# Set up Google Cloud configuration for Gemini (with Vertex AI)
# MODEL_PROVIDER = "gemini"
# MODEL = "gemini-2.0-flash-001"
# config.set_google_cloud_config("YOUR PROJECT", "YOUR LOCATION")

# Set up GPT configuration
MODEL_PROVIDER = "gpt"
MODEL = "gpt-4o-mini"
config.set_api_key("gpt", "YOUR API KEY")

In [4]:
### Define your own formatter function for your data
def format_friends_transcript_from_convokit(convo):
    utt_lst = convo.get_utterance_ids()
    speaker_ids = {}
    transcript = ""
    for utt_id in utt_lst:
        utt = corpus.get_utterance(utt_id)
        if "TRANSCRIPT_NOTE" not in utt.speaker.id:
            if utt.speaker.id not in speaker_ids:
                speaker_ids[utt.speaker.id] = 1 + len(speaker_ids)
            transcript += "Speaker"+str(speaker_ids[utt.speaker.id]) + " : " + utt.text+ "\n\n"
    return transcript

In [5]:
### Prepare your own prompt for writing the SCD with your data
friends_summary_prompt = """
Write a short summary capturing the trajectory of a casual conversation. 
Do not include specific topics, events, or arguments from the conversation. The style you should avoid is illustrated in 
Example Sentence 1: “Speaker1 said they had a difficult day at work, and mentioned that their boss was unfair. Speaker2 listened and agreed that bosses can be tough, then suggested they go out for dinner to forget about it..” Instead, you should include indicators of sentiments (e.g., warmth, empathy, humor, nostalgia, vulnerability, support), individual intentions (e.g., building rapport, offering reassurance, seeking validation, self-disclosure, active listening, gentle disagreement, creating distance), and conversational strategies (if any) such as “collaborative storytelling,” “inside jokes,” “mirroring emotions,” and “affectionate teasing.” 
The following sentences demonstrate the style you should follow: 
Example Sentence 2: “Both speakers have similar feelings and appeared mutually supportive. Speaker1 initiates with a moment of self-disclosure, and Speaker2 responds with empathy and validation. Both speakers build on this exchange, strengthening their rapport.” 
Example Sentence 3: “The two speakers connected with back-and-forth affectionate teasing. Throughout the conversation, they kept building on each other's humor with playful remarks, creating a lighthearted and comfortable discussion.” Overall, the trajectory summary should capture the key moments where the emotional connection of the conversation notably changes. Here is an example of a complete trajectory summary: The conversation begins with two speakers exchanging neutral, surface-level comments. Speaker1 then shifts the tone by sharing a personal anecdote, prompting Speaker2 to respond with warmth and empathy. Speaker1 elaborates on their story and their need, but Speaker2 does not extend their support but retracts it. 
Now, provide the trajectory summary for the following conversation. 
Conversation Transcript: {formatted_object}. 
Now, summarize this conversation. Remember, do not include specific topics, claims, or arguments from the conversation. Instead, try to capture the speakers' sentiments, intentions, and conversational/persuasive strategies. Limit the trajectory summary to 80 words. 
Trajectory Summary:"""

In [6]:
friends_sop_prompt = """
Here is a trajectory summary of a conversation that lays out how the dynamics of the conversation developed. You need to parse the summary into events in order. 
Follow the following guidelines:
1. Try to maintain the original language of the summary as much as you can. 
2. Provide your output as a Python dictionary with the following structure:
_(Note: Do NOT use markdown, JSON formatting, or code block delimiters.)_ 
{{
    '0': "" // description of the event
    '1': ...
    ...
}}
Here is the summary:
{formatted_object}
"""

Initialize your SCD transformer

In [7]:
scd_transformer = SCD(
        model_provider=MODEL_PROVIDER,
        config=config,
        model=MODEL,
        custom_scd_prompt=friends_summary_prompt,
        custom_sop_prompt=friends_sop_prompt,
        custom_prompt_dir="friends_prompts",
        generate_scd=True,
        generate_sop=True,
        scd_metadata_name="machine_scd",
        sop_metadata_name="machine_sop",
        conversation_formatter=format_friends_transcript_from_convokit
    )

In [8]:
conversation_ids = list(corpus.get_conversation_ids())[:2]
selector = lambda conv: conv.id in conversation_ids

In [9]:
corpus = scd_transformer.transform(corpus, selector=selector)

In [10]:
convo = corpus.get_conversation(conversation_ids[0])
print("SCD: ", convo.meta["machine_scd"])
print("SoP: ", convo.meta["machine_sop"])

SCD:  The conversation begins with playful teasing and lighthearted banter, creating a warm atmosphere. As one speaker expresses vulnerability, others respond with empathy and support, fostering a sense of camaraderie. The tone shifts to deeper emotional revelations, with moments of humor interspersed, allowing for self-disclosure and connection. Despite some tension, the group maintains a supportive dynamic, ultimately reinforcing their bonds through shared experiences and gentle encouragement, culminating in a mix of nostalgia and understanding.
SoP:  {
    '0': "The conversation begins with playful teasing and lighthearted banter, creating a warm atmosphere.",
    '1': "As one speaker expresses vulnerability, others respond with empathy and support, fostering a sense of camaraderie.",
    '2': "The tone shifts to deeper emotional revelations, with moments of humor interspersed, allowing for self-disclosure and connection.",
    '3': "Despite some tension, the group maintains a suppo

## Compute ConDynS Score


In [11]:
condyns = ConDynS(model_provider=MODEL_PROVIDER, 
                  model=MODEL, 
                  config=config)

In [ ]:
convo_id1 = conversation_ids[0]
convo_id2 = conversation_ids[1]

# Compare conversations
result, condyns_score = condyns.compare_conversations(
    corpus=corpus,
    convo_id1=convo_id1, 
    convo_id2=convo_id2,
    sop_meta_name="machine_sop",
    formatter=format_friends_transcript_from_convokit  # Use our custom formatter
)

print(f"ConDynS Score between conversations {convo_id1} and {convo_id2}: {condyns_score}")

convo1 = corpus.get_conversation(convo_id1)
convo2 = corpus.get_conversation(convo_id2)

score_key1 = f"condyns_{convo_id1}_{convo_id2}"
result_key1 = f"condyns_result_{convo_id1}_{convo_id2}"
score_key2 = f"condyns_{convo_id2}_{convo_id1}"
result_key2 = f"condyns_result_{convo_id2}_{convo_id1}"
print(f"Score stored in conversation {convo_id1} metadata: {convo1.meta.get(score_key1)}")
print(f"Score stored in conversation {convo_id2} metadata: {convo2.meta.get(score_key2)}")

print(f"Score reasoning stored in conversation {convo_id1} metadata: {convo1.meta.get(result_key1)}")
print(f"Score reasoning stored in conversation {convo_id2} metadata: {convo2.meta.get(result_key2)}")

ConDynS Score between conversations s01_e01_c01_u001 and s01_e01_c02_u001: 0.6499999999999999
Score stored in conversation s01_e01_c01_u001 metadata: 0.6499999999999999
Score stored in conversation s01_e01_c02_u001 metadata: 0.6499999999999999
Score reasoning stored in conversation s01_e01_c01_u001 metadata: [{'0': {'analysis': 'Transcript starts with playful banter but lacks warmth.', 'score': 0.3}, '1': {'analysis': 'Some expressions of vulnerability are present, but empathy is minimal.', 'score': 0.4}, '2': {'analysis': 'Emotional revelations occur, but humor is not well interspersed.', 'score': 0.5}, '3': {'analysis': 'Tension is present, but support is inconsistent.', 'score': 0.4}, '4': {'analysis': 'Shared experiences are mentioned, but bonds are not strongly reinforced.', 'score': 0.3}}, {'0': {'analysis': 'Transcript starts with playful banter about dating, matching the first event.', 'score': 1}, '1': {'analysis': 'Speaker6 expresses vulnerability about a breakup, aligning wi